# CT Visualization Demo

This notebook demonstrates how to:

- Load a CT series from disk (NIfTI or DICOM directory)
- Extract axial/coronal/sagittal slices
- Apply simple window/leveling for display
- Show basic metadata alongside the image

It uses `src/visualization/ct_viewer.py` from the main project.


In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parents[1]))  # parent of notebooks/


import matplotlib.pyplot as plt
import numpy as np

from src.visualization.ct_viewer import (
    load_ct_series,
    get_slice,
    apply_window,
    extract_display_metadata,
)


In [ ]:
# Example: NIfTI file
ct_path = Path("../../data/raw_test/Task06_Lung/imagesTr/lung_001.nii.gz")

# Example: DICOM directory
# ct_path = Path("path/to/dicom_directory")

ct = load_ct_series(ct_path)
display_meta = extract_display_metadata(ct)

print("CT summary:")
for k, v in display_meta.items():
    print(f"{k}: {v}")


In [ ]:
z, y, x = ct.shape
print(f"Volume shape (z, y, x): {ct.shape}")

# Choose indices for each plane
axial_idx = z // 2
coronal_idx = y // 2
sagittal_idx = x // 2

axial_slice = get_slice(ct, axial_idx, plane="axial")
coronal_slice = get_slice(ct, coronal_idx, plane="coronal")
sagittal_slice = get_slice(ct, sagittal_idx, plane="sagittal")


In [ ]:
# Lung window as a reasonable default
LUNG_CENTER = -600.0
LUNG_WIDTH = 1500.0

axial_win = apply_window(axial_slice, center=LUNG_CENTER, width=LUNG_WIDTH)
coronal_win = apply_window(coronal_slice, center=LUNG_CENTER, width=LUNG_WIDTH)
sagittal_win = apply_window(sagittal_slice, center=LUNG_CENTER, width=LUNG_WIDTH)


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(axial_win, cmap="gray", origin="lower")
axes[0].set_title(f"Axial (z={axial_idx})")
axes[0].axis("off")

axes[1].imshow(coronal_win, cmap="gray", origin="lower")
axes[1].set_title(f"Coronal (y={coronal_idx})")
axes[1].axis("off")

axes[2].imshow(sagittal_win, cmap="gray", origin="lower")
axes[2].set_title(f"Sagittal (x={sagittal_idx})")
axes[2].axis("off")

plt.tight_layout()
plt.show()


In [ ]:
import ipywidgets as widgets
from IPython.display import display

z, _, _ = ct.shape

def show_axial(idx: int):
    slice_2d = get_slice(ct, idx, plane="axial")
    img = apply_window(slice_2d, center=LUNG_CENTER, width=LUNG_WIDTH)

    plt.figure(figsize=(5, 5))
    plt.imshow(img, cmap="gray", origin="lower")
    plt.title(f"Axial slice {idx} / {z-1}")
    plt.axis("off")
    plt.show()

slider = widgets.IntSlider(min=0, max=z - 1, step=1, value=z // 2, description="z")
widgets.interact(show_axial, idx=slider)
display(slider)
